---
title: "Análisis de Modelos y Validación"
theme: Montpellier
author: "Máster en Ciencias de Datos e Ingeniería de Computadores, Minería de Datos - Preprocesamiento y clasificación"
date: 29 Octubre 2024
toc: true
toc-title: Tabla de Contenidos
toc-depth: 1
execute:
  echo: true
output:
  beamer_presentation:
    slide_level: 1
format:
  html:
    code-fold: false
    code-summary: "Muestra código"
    fig-width: 5
    fig-height: 3
    fig-align: left
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: dark
    fig-align: left
    fig-height: 5
    fig-cap-location: margin
    smaller: true
---

## Imports iniciales

Vamos a importar las bibliotecas principales que usaremos en este guión.

In [ ]:
# Essential imports: sklearn with version 0.23 or later and numpy with version 1.19 or later
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns

from warnings import simplefilter
#simplefilter(action='ignore', category=FutureWarning) #Evitar los warnings de sklearn con KNN

## Lo que veremos

::: {.notes}

Aunque se ha tratado en otras asignaturas del máster, en esta práctica revisitaremos las técnicas de validación, evaluación y métricas en las principales bibliotecas de Python para Data Science.
Tomaremos un conjunto clásico de ejemplo para regresión, y produciremos diferentes esquemas de evaluación y métricas interesantes.
De igual forma, avanzando el resto de guiones que tendremos sobre clasificación, realizaremos un ejemplo similar para clasificación en el que nos centraremos en las métricas de clasificación.

:::

- Repasaremos las técnicas de validación, evaluación y métricas en las principales bibliotecas de Python para Data Science.

- Dados ejemplos clásicos, veremos métricas, y produciremos distintos esquemas de evaluación.

- Veremos tanto regresión como clasificación.

- Es aplicable al resto de prácticas.

# Métricas de evaluación para regresión

## Métricas de evaluación para regresión

Para poder estimar la bondad de las predicciones, tendremos que evaluar algunas métricas que permitan comparar las predicciones de cada ejemplo que hemos proporcionado a la función predict frente a su salida real.

Vamos a repasar las principales incluidas en sklearn [https://scikit-learn.org/stable/modules/model_evaluation.html]([https://scikit-learn.org/stable/modules/model_evaluation.html).

El **error absoluto medio (MAE)** es la medida de evaluación más sencilla, y no es tan popular como el RMSE. 

$$ MAE=\displaystyle \frac{1}{N} \sum_{i=1}^N |y_i-\hat{y_i}| $$
$$ MAPE=\displaystyle \frac{1}{N} \sum_{i=1}^N \frac{|y_i-\hat{y_i}|}{max(\epsilon, |y_i|)} $$

donde $y_i$ es la etiqueta real y $\hat{y_i}$ la etiqueta predicha por el modelo para el dato $i$.

---

Otra medida muy usada es el **error cuadrático medio (MSE)**. 

$$ MSE=\displaystyle \frac{1}{N} \sum_{i=1}^N (y_i-\hat{y_i})^2 $$

El error cuadrático medio (MSE) es más sensible a outliers comparando con el MAE. Una forma de suavizar el MSE es utilizar **la Raíz del MSE (RMSE)**:

$$ RMSE=\displaystyle \sqrt {\frac{1}{N} \sum_{i=1}^N (y_i-\hat{y_i})^2} $$

Las métricas anteriores generan valores en un rango que depende del problema, lo que dificulta la comparación entre problemas diferentes. 

---

**El R^2**

El coeficiente R<sup>2</sup> establece un valor acotado entre 0 y 1. En particular, explica hasta qué punto las variables de entrada explican la **variación** de la variable de salida. La definición de R<sup>2</sup> es:

La suma de residuos: $SS_{res} =  (y_i - \hat{y_i})$ y el cuadrado de la distancia: $SS_{tot} = \displaystyle \sum_i (y_i - \hat{y_i})^2$

$$ R^2=\displaystyle 1-\frac{SS_{res}}{SS_{tot}}=1-\frac{\sum_i (y_i - \hat{y_i})^2}{\sum_i (y_i - \bar{y_i})^2} $$

Algunas particularidades:

+ Cuanto más cercano a 1, mejor. 
+ Un valor negativo es posible, si la predicción es peor que la media.
 
-----------------------------------------------------------------------------------------------

R<sup>2</sup> tiene limitaciones: añadir más variables siempre hará que R<sup>2</sup> se incremente, aunque las variables añadidas **no** aporten información sobre la variable de salida. Existe una alternativa R<sup>2</sup> ajustada que no tiene estos problemas.

$$ \bar{R^2}=1-\displaystyle (1-R^2)\cdot \frac{(N-1)}{N-p-1}$$

donde $p$ es el número de variables (excepto la que se predice).

## Ejemplo de datos

In [ ]:
#Vamos a generar un pequeño modelo de regresión sobre
# el conjunto diabetes, para examinar sus resultados
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

reg = LinearRegression()
reg.fit(X, y)

y_pred = reg.predict(X)
print(y_pred[:5])
print(y[:5])

En *y_pred* tendremos las predicciones generadas por el modelo, gracias a la función *predict* (más sobre esto en la segunda parte del guión).

---

Ahora aplicamos las distintas métricas

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error, mean_squared_error,   root_mean_squared_error, r2_score

results = {}
results['MAE'] = mean_absolute_error(y, y_pred)
results['MAPE'] = mean_absolute_percentage_error(y, y_pred)
results['MSE'] = mean_squared_error(y, y_pred)
results['RMSE'] = root_mean_squared_error(y, y_pred)
results['R2'] = r2_score(y, y_pred)

for metric, value in results.items():
    print(f"{metric : >4}: {value : 10.4f}")

# Métricas de evaluación para clasificación
## Métricas de evaluación para clasificación

De forma breve, vamos a repasar las métricas de evaluación para clasificación más populares. 

### Datos de Ejemplo 

Vamos a usar el conjunto breastCancer utilizando KNN, visto ya en asignaturas previas.

Vamos a "crear" un KNN sobre el conjunto completo.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier

breastCancer = sklearn.datasets.load_breast_cancer()
X = breastCancer.data
y = breastCancer.target

knn = KNeighborsClassifier(n_neighbors=5)
knn_model = knn.fit(X, y)

knn_predictions = knn_model.predict(X)

## Métricas de clasificación
 
Para evaluar modelos de clasificación, las métricas no pueden ser las mismas que los modelos de regresión. Las etiquetas son nominales (texto o índices de texto) y no podemos hacer cálculos sobre ellas (restas, cuadrados, etc.)

Las métricas son muy diferentes, no tiene sentido operar con categorías.

Debemos tener en cuenta la matriz de confusión, que generará las diferentes métricas de evaluación. Recordémosla:

::: {.columns}

:::: {.column width="40%"}
![Matriz de confusión](./img/cm.PNG){height="160"}
::::

:::: {.column width="40%"}
+ FP: Falso Positivo.
+ FN: Falso Negativo.
+ TP: Verdadero Positivo.
+ FP: Falso Positivo.
::::

:::

Vamos a definir algunas métricas.

---

### Accuracy

La precisión (**Accuracy**) se define como la relación entre el número de predicciones correctas y el número total de predicciones. Se sitúa entre [0,1]. En general, una mayor precisión significa un mejor modelo (TP y TN deben ser altos).

$$ Accuracy = \displaystyle \frac{TP+TN}{TP+FP+TN+FN}$$

El **Accuracy** tiene problemas con el desequilibrio de la cantidad de ejemplos de diferente clases. Por este motivo, aparecen otras métricas como Precision y Recall. 

---

### Recall

$$ Recall = \frac{TP}{TP+FN}$$

El **Recall** es una métrica útil en el caso de la detección del cáncer, donde queremos minimizar el número de falsos negativos para cualquier uso práctico, ya que no queremos que nuestro modelo marque como seguro a un paciente que padece cáncer. 

Por otro lado, predecir un paciente sano como canceroso no es un gran problema ya que, en un diagnóstico posterior, se aclarará que no tiene cáncer. 

El Recall también se conoce como Sensibilidad.

---

### Precisión

$$ Precision = \displaystyle \frac{TP}{TP+NP}$$

**Precision** es útil cuando queremos reducir el número de falsos positivos. Consideremos un sistema que predice si el correo electrónico recibido es spam o no. Tomando el spam como clase positiva, no queremos que nuestro sistema prediga como spam los correos electrónicos que no son spam (correos importantes), es decir, el objetivo es reducir el número de falsos positivos.

---

### Specifity

$$ Specifity = \displaystyle \frac{TN}{TN+FP} $$

La especificidad (**Specifity**) se define como la relación entre TN y TN + FP. Queremos que el valor de la especificidad sea alto. 

Su valor se sitúa entre [0,1]. 

Si nos fijamos, es la definición de **Recall** para la clase negativa.

---

### F1-score

$$ F1-score =  \displaystyle 2 \cdot \frac{precision \cdot recall}{precision + recall} =\frac{2\cdot TP}{2\cdot TP + FP + FN}$$

El **F1-score** es una métrica que combina Precision y Recall y equivale a la media armónica de la precisión y la recuperación. 

Es más adecuado en clases menos balanceadas.

--- 

### Ejemplo de uso

Vamos a usar las rutinas de sklearn para obtener las medidas anteriores.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import precision_score, recall_score, classification_report

print("Accuracy: ", accuracy_score(y, knn_predictions))
#El método score es equivalente a accuracy_score en este caso
print("Con el método score: ", knn.score(X, y)) 
print("Recall: ", recall_score(y, knn_predictions))
print("Precision: ", precision_score(y, knn_predictions))
print("F1 score: ", f1_score(y, knn_predictions))

#Para Specifity, podemos indicar que la etiqueta positiva es 0 (si anteriormente era 1)
print("Specifity: ", recall_score(y, knn_predictions, pos_label=0))

# Con classification_report se puede de obtener un informe de todas las métricas principales
# Para la fila de la clase 0, el valor de recall es el mismo que el de specifity obtenido anteriormente
print("Informe completo\n",classification_report(y, knn_predictions))

## Área bajo la curva ROC (AUC-ROC)

Una forma de comparar el rendimiento de un modelo frente a ambas clases es el área bajo la curva ROC. Enfrenta, en un gráfico, el valor de $FPR (False Positive Rate)$ y el $TPR (True Positive Rate)$. TPR y FPR se definen así:

$$ TPR=\displaystyle \frac{TP}{TP+FN}, FPR=\displaystyle \frac{FP}{TP+FN} $$

El área bajo la curva resultante es un resumen del rendimiento en estas dos características, con un valor en el intervalo [0,1]. Cuanto más cercano a 1, mejor. Un valor de 0,5 se considera equivalente a un predictor completamente aleatorio (e inútil).

## Área bajo la curva

Se pueden comparar algoritmos mostrando la curva ROC.

![Ejemplos de curvas ROC y su utilidad](./img/aucroc_algs.png){width="50%"}

--- 

Se pueden generar fácilmente con:

In [ ]:
#| code-fold: true
from sklearn.metrics import roc_auc_score, roc_curve, auc

#Vamos a imprimir el área bajo la curva ROC para el modelo KNN
print(roc_auc_score(y, knn_predictions))

#Podemos obtener los valores de TPR y FPR usando la función roc_curve
fpr, tpr, thresholds = roc_curve(y, knn_predictions)
print("TPR: ", tpr)
print("FPR: ", fpr)
#Podemos calcular el área bajo la curva ROC usando la función auc y los valores de TPR y FPR
print(auc(fpr, tpr))

#A continuación, vamos a mostrar la figura de la curva ROC
plt.figure()
f = sns.relplot(x=fpr, y=tpr, color='red', label=f"ROC curve (area = {auc(fpr, tpr):0.2f})", kind="line", aspect=2)
f.set(xlabel="False Positive Rate", ylabel="True Positive Label")

----

No se puede aplicar con varias clases, pero hay alternativas:

- Aplicar para predecir una clase frente al resto.

- Aplicar un promedio para cada métrica ($micro-average ROC$):

$$ TPR=\displaystyle \frac{\sum_c TP_c}{\sum_c (TP_{c}+TN_{c})}, FPR=\frac{\sum_c FP_c}{\sum_c (TP_{c}+TN_{c})}  $$

![Ejemplo con varias clases](./img/aucroc_class.png){width="10%"}
 
 ---
 
Ambos se puede aplicar fácilmente con [Yellowbrick](https://www.scikit-yb.org/en/latest/api/classifier/rocauc.html?highlight=roc#module-yellowbrick.classifier.rocauc)

In [ ]:
#| code-fold: true
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier

from yellowbrick.classifier import ROCAUC

# Load the classification dataset
df = load_wine()
X = df.data
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Instantiate the visualizer with the classification model
model = DecisionTreeClassifier()
visualizer = ROCAUC(model)

visualizer.fit(X_train, y_train)        # Fit the training data to the visualizer
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

# Esquemas de validación y particionamiento

## Esquemas de validación y particionamiento

Hasta ahora, hemos generado un modelo sobre el conjunto completo y evaluado el rendimiento sobre el propio conjunto de entrenamiento (el conjunto completo).
Como hemos visto en asignaturas previas, no es una buena idea:

1. Hay un gran sesgo, ya que el modelo predice sobre los ejemplos que conoce.
2. Desconocemos su capacidad de tratar con ejemplos nunca vistos (habilidad de generalización del modelo).
3. Cualquier ajuste de parámetros producirá un sobreajuste seguramente.

Es necesario realizar algún tipo de partición para evitar estos problemas. Los diferentes tipos de particionamiento del conjunto completo para poder validar correctamente el rendimiento del modelo generado se les conoce como esquemas de validación o particionamiento.

## Hold-out

Vamos a crear un esquema de validación Hold-out, el más simple. Partimos aleatoriamente el conjunto en 2 partes, de tamaños arbitrarios. Una parte será el conjunto de entrenamiento y otro el de test.

Utilizamos la función [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Juega con diferentes tamaños de partición para observar su efecto en las diferentes métricas.

---

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Por si acaso, vamos a cargar el conjunto de datos de cancer de mama de nuevo
X = breastCancer.data
y = breastCancer.target

#Dividimos el conjunto de datos en un conjunto de entrenamiento y otro de test: 20% para test, y 80% para entrenamiento
#No queremos que los datos se desordenen respecto a su orden inicial, por lo que indicamos shuffle=false
#Ojo con el parámetro shuffle: https://scikit-learn.org/stable/modules/cross_validation.html#a-note-on-shuffling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False);

knn = KNeighborsClassifier(n_neighbors=5)
#OJO: ahora entrenamos el modelo con los datos de entrenamiento
knn_model = knn.fit(X_train, y_train)

#Y ahora obtenemos las predicciones para los datos de test
knn_predictions = knn_model.predict(X_test)

#Vamos a mostrar las métricas de rendimiento sobre el conjuto de test Y_test
print("Informe completo\n",classification_report(y_test, knn_predictions))

## Repeated Hold-out

Uno de los principales problemas del Hold-out es la dependencia de los buenos (o malos) resultados de nuestro modelo a la elección aleatoria de los datos que pertenecen al conjunto de entrenamiento y de test.

Para disminuir el efecto de esta aleatoriedad, podemos realizar varios de forma repetida, de forma que, en promedio, los ejemplos difíciles estarán alternativamente en el entrenamiento o en el test. A este esquema se le conoce como **Repeated Hold-out**.

Para simularlo (Scikitlearn no tiene una función específica), usamos ShuffleSplit para realizar tantos splits (repeticiones) como queramos de un Hold-out.

Para nuestro ejemplo, recorremos con un bucle cada partición/split y ajustamos el modelo KNN para ver el resultado.

---

In [ ]:
from sklearn.model_selection import ShuffleSplit

#Con n_splits controlamos cuántas veces queremos repetir el Hold-out. El parámetro random_state actúa como la semilla para la generación de las particiones
repeatedHoldOut = ShuffleSplit(n_splits=10, test_size=0.25, random_state=100)
knn = KNeighborsClassifier(n_neighbors=5) #Vamos a usar el mismo modelo que antes

breastCancer = load_breast_cancer()
X = breastCancer.data
Y = breastCancer.target

for train_index, test_index in repeatedHoldOut.split(X):
    X_train, Y_train = X[train_index], Y[train_index]    
    X_test, Y_test = X[test_index], Y[test_index]
    knn.fit(X_train,Y_train)
    prediccion = knn.score(X_test, Y_test)
    print("Accuracy: ", prediccion)
    
#Ejercicio: Calcular la media de los valores de accuracy obtenidos en el bucle anterior

## Validación cruzada

::: {.notes}
Un problema que tiene el Hold-out repetido radica en la falta de independencia de las particiones realizadas: los ejemplos pueden encontrarse repetidamente en entrenamiento y test, lo que hace que los valores de test calculados no tengan *independencia*.
:::

Problema: Falta de independencia de las particiones.

::: {.notes}
Para garantizar la independencia (y por tanto, su generalidad) de las particiones de test, sería importante realizar la partición repetida del Hold-out de forma que cada ejemplo sólo pueda estar una vez en una partición de test.
:::

Se hace necesario repetir las particiones, pero de forma más sistemática:

Este concepto es el que propone la **Validación cruzada**. Se determina el número de conjuntos de test que se crearán y cada ejemplo sólo podrá aparecer en uno de dichos conjuntos de test. Las particiones de entrenamiento asociadas a cada conjunto de test contendrán todos los ejemplos que no están en dicho conjunto de test. Por lo tanto, para $K$ conjuntos de test, cada ejemplo estará en $K-1$ conjuntos de entrenamiento.

![Validación cruzada](./img/Cross_Validation.jpg)

---

Mostrar un ejemplo de cómo realizar una validación cruzada en sklearn, usando [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

In [ ]:
#En primer lugar, vamos a realizar el trabajo de particionar y entrenar el modelo con el conjunto de datos de entrenamiento para cada split/fold de forma manual
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report

# KFold permite realizar particiones de los datos en K folds
kf5 = KFold(n_splits=5, shuffle=False)
kf3 = KFold(n_splits=3, shuffle=False)

#Cargamos Iris
iris = load_iris(); X = iris.data; y = iris.target

#La función split de KFold devuelve los índices de los datos de entrenamiento y test para cada fold
#Siendo un generator, podemos iterar sobre él para obtener los índices de cada fold
#Será necesario aplicar los índices sobre el conjunto X y el conjunto y para obtener los datos de entrenamiento y test para cada fold
run = 1
for train_index, test_index in kf5.split(X,y):
    X_train = X[train_index]
    y_train = y[train_index]
    
    X_test = X[test_index]
    y_test = y[test_index]
    
    knn.fit(X_train,y_train)
    prediccion = knn.score(X_test, y_test)
    print(f"Accuracy[{run}]: {prediccion:.2f}")
    run += 1

#Normalmente, se hará la media del Accuracy de los diferentes fold

---

## Uso de shuffle

Ahora vamos a mostrar el uso de la opción *shuffle*. Si no la activamos, los datos se dividirán en los folds de forma secuencial, en el orden original del conjunto.

Vemos qué pasa si no se desordena:

In [ ]:
#| code-fold: true 
#Vamos a examinar cómo están los datos originalmente en Iris (cargamos iris como un DataFrame para poder tener los nombres de las clases)
iris_df = pd.DataFrame(data=iris.data,columns=iris.feature_names)

features = iris['feature_names'] #Obtenemos los nombres de las columnas
iris_df['target'] = iris.target #Añadimos la columna target
iris_df["target_name"] = iris_df['target'].map({i:name for i,name in enumerate(iris.target_names)}) #Añadimos la columna target_name para tener los nombres de las clases

target_name = iris_df["target"]

fig, ax = plt.subplots(1,3, figsize=(13,3), sharey=True)
for i, (train_index, test_index) in enumerate(kf3.split(iris_df)): #Ojo como utilizamos el K-Fold de 3 folds para iterar sobre el DataFrame y obtener las 3 clases de forma secuencial, ya que Iris tiene 3 clases de 50 ejemplos cada una...
    ax[i].scatter(x=train_index,y=target_name.iloc[train_index],label ="train", c='b')
    ax[i].scatter(x=test_index,y=target_name.iloc[test_index], label = "test", c='r')
    ax[i].set_title(f"Fold {i+1}")

ax[0].set_yticks([0,1,2])
ax[0].set_yticklabels(iris["target_names"])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

---

## No aprende bien

En este ejemplo, cada conjunto de entrenamiento contiene solo una clase, por lo que no aprende bien.

::: {.notes}
Si ahora entrenamos sobre los datos de entrenamiento de cada fold con shuffle=False y obtenemos el accuracy sobre los datos de test, veremos que el accuracy es cero, ya que no hay ejemplos de las otras clases en cada partición de entrenamiento
:::

In [ ]:
i = 1
for train_index, test_index in kf3.split(iris_df):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index][features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index]['target']
        
    #Entrenamos y evaluamos
    knn.fit(X_train, y_train) #Training the model
    print(f"Accuracy para el fold de TEST {i}: {accuracy_score(y_test, knn.predict(X_test)):.2f}")
    i += 1


## Usando Shuffle

Para evitar este tipo de problemas, debemos utilizar la opción shuffle=True (es bueno fijar semilla por reproducibilidad):

In [ ]:
#| code-fold: true 
kf = KFold(n_splits=3, shuffle=True, random_state=100)

i = 1
for train_index, test_index in kf.split(iris_df):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index].loc[:,features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index].loc[:,'target']
    
    #Entrenamos y evaluamos
    knn.fit(X_train, y_train) #Training the model
    print(f"Accuracy para el fold de TEST {i}: {accuracy_score(y_test, knn.predict(X_test))}")
    i += 1
    
#Vamos a ver cómo se distribuyen los datos en cada fold
plt.scatter(x=y_train.index,y=iris_df.iloc[train_index].loc[:,'target_name'],label ="train")
plt.scatter(x=y_test.index,y=iris_df.iloc[test_index].loc[:,'target_name'], label = "test")
plt.legend()

## Clases Desbalanceadas

Además de la importancia de la opción *shuffle* para la estratificación, existen problemas desequilibrados en los que el simple muestreo puede generar particiones de test aún más desequilibradas o incluso sin algunos ejemplos de ciertas clases.

---

In [ ]:
#| code-fold: true 
kf = KFold(n_splits=3, shuffle=True, random_state=100)

fold = 1
plot_df = pd.DataFrame({'type':  [], 'class': [], 'count': [], 'fold': []})
target_names = iris_df.target_name.unique()
targets = iris_df.target.unique()

for train_index, test_index in kf.split(iris_df, iris_df["target"]):
    for (target, target_name) in zip(targets, target_names):
        count = (iris_df.loc[train_index, 'target'] == target).sum()
        df_row = pd.DataFrame([['train', target, count, fold]], columns=['type', 'class', 'count', 'fold'])
        plot_df = pd.concat([plot_df, df_row])
        count = (iris_df.loc[test_index, 'target'] == target).sum()
        df_row = pd.DataFrame([['test', target, count, fold]], columns=['type', 'class', 'count', 'fold'])
        plot_df = pd.concat([plot_df, df_row])

    fold += 1

sns.catplot(data=plot_df, x='type', y='count', hue='class', col='fold', kind='bar', legend=False)
plt.legend(title="Class", labels=target_names,
           bbox_to_anchor= (1.02,0.5))
plt.show()

---

Una forma de evitar esto es usar el K-Fold **estratificado**, el cual genera folds o splits que mantienen la misma proporción de ejemplos del dataset original.

---

In [ ]:
#| code-fold: true 
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=100)
i = 1
for train_index, test_index in kf.split(iris_df, iris_df["target"]):
    X_train = iris_df.iloc[train_index].loc[:, features]
    X_test = iris_df.iloc[test_index].loc[:,features]
    y_train = iris_df.iloc[train_index].loc[:,'target']
    y_test = iris_df.loc[test_index].loc[:,'target']

    #Entrenamos y evaluamos
    knn.fit(X_train, y_train) #Training the model
    print(f"Accuracy para el fold de TEST {i}: {accuracy_score(y_test, knn.predict(X_test))}")
    i += 1
    
#Observa los resultados en comparación al Kfold anterior con shuffle: el comportamiento en las diferentes particiones es mucho más homogéneo

plt.scatter(x=y_train.index,y=iris_df.iloc[train_index].loc[:,'target_name'],label ="train")
plt.scatter(x=y_test.index,y=iris_df.iloc[test_index].loc[:,'target_name'], label = "test")
plt.legend()
plt.title("K-fold estratificado para el último fold")
plt.show()

---

In [ ]:
#| code-fold: true 
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=100)

fold = 1
plot_df = pd.DataFrame({'type':  [], 'class': [], 'count': [], 'fold': []})
target_names = iris_df.target_name.unique()
targets = iris_df.target.unique()

for train_index, test_index in kf.split(iris_df, iris_df["target"]):
    for (target, target_name) in zip(targets, target_names):
        count = (iris_df.loc[train_index, 'target'] == target).sum()
        df_row = pd.DataFrame([['train', target, count, fold]], columns=['type', 'class', 'count', 'fold'])
        plot_df = pd.concat([plot_df, df_row])
        count = (iris_df.loc[test_index, 'target'] == target).sum()
        df_row = pd.DataFrame([['test', target, count, fold]], columns=['type', 'class', 'count', 'fold'])
        plot_df = pd.concat([plot_df, df_row])

    fold += 1

sns.catplot(data=plot_df, x='type', y='count', hue='class', col='fold', kind='bar', legend=False)
plt.legend(title="Class", labels=target_names,
           bbox_to_anchor= (1.02,0.5))
plt.show()

## Validación Cruzada de forma más fácil

Puesto que realizar el K-fold es habitual y utilizar los índices para luego acceder a nuestro Dataframe o Array genera mucho código repetido, Scikit learn proporciona la función *cross_validate* que realiza la misma tarea en una única llamada. A cambio, perdemos el control de los índices generados y cualquier posibilidad de análisis forense posterior.

In [ ]:
from sklearn.utils import shuffle

cv = cross_validate(knn, X, y, cv=3)

#Vamos a ver los resultados en test
for i, score in enumerate(cv['test_score']):
    print(f"Accuracy en el fold {i+1}: {score}")

# Existe la posibilidad de usar un objeto KFold definido previamente
# Es la forma de controlar las opciones de shuffle y random_state entre otras
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=100) #Prueba a jugar con los argumentos aquí
cv = cross_validate(knn, X, y, cv=kf)
for i, score in enumerate(cv['test_score']):
    print(f"Accuracy en el fold {i+1} con un objeto kf: {score}")

Si no queremos todas las métricas, podemos utilizar un objeto de tipo cross_val_score para obtener solo el accuracy, por ejemplo

In [ ]:
from sklearn.metrics import make_scorer #Mal sitio para el import, pero bueno

cv = cross_val_score(knn, X, y, cv=3)
#print(cv) #Vemos que es un array, ya no es un dataframe como en el caso anterior con diferentes medidas
for i, score in enumerate(cv):
    print(f"Accuracy en el fold {i+1}: {score}")
    
#Vamos a probar cross_val_score con otra medida de rendimiento, el F1-score
#Debemos crear un objeto scorer con la medida que nos interesa previamente
f1=make_scorer(f1_score, average='weighted')
cv = cross_val_score(knn, X, y, cv=3, scoring = f1)

for i, score in enumerate(cv):
    print(f"F1 en el fold {i+1}: {score}")

## Leave-one out

Un caso extremo de K-fold cross validation es cuando el número de folds/splits coincide con el número de ejemplos del dataset completo. En este caso, cada ejemplo actúa como una partición de test. De forma alternativa, todos los ejemplos menos uno se combinan para generar un conjunto de entrenamiento y se prueban sobre el ejemplo que se ha dejado fuera.

Tiene especial relevancia para modelos Lazy (como KNN), cuando queremos evaluar su comportamiento sobre el conjunto completo sin incurrir en un error de sesgo.

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
i=1

for train_index, test_index in loo.split(X):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
   # print(X_train, X_test, y_train, y_test)
    
     #Entrenamos y evaluamos
    knn.fit(X_train, y_train) #Training the model
    print(f"Accuracy para el ejemplo de TEST {i}: {accuracy_score(y_test, knn.predict(X_test))}")
    i += 1

# Ejercicios propuestos
1. Calcule el valor de R<sup>2</sup> ajustado según la equación proporcionada y compare su valor con R<sup>2</sup>.
2. Manipule el número de vecinos de KNN para observar cómo afecta al rendimiento del modelo. ¿Qué ocurreo con Precision y Recall cuando aumenta el número de vecinos?
3. Hemos visto lo que ocurre con un problema de clasificación de 2 clases (Binario). Es normal encontrarnos con problemas de varias clases (multiclase). Visite https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules y en la sección 3.3.2.1 lea detenidamente cómo se puede agregar los valores de Precision, Recall, etc. para varias clases. Cargue un conjunto multiclase (por ej. Iris) y obtenga las métricas de rendimiento vistas para este conjunto multiclase, jugando con los diferentes tipos de promediado para observar las diferencias (si el problema escogido está desequilibrado, se observarán mejor).
4. Pruebe los diferentes esquemas de validación y diferentes métricas para otro problema diferente a Iris (https://scikit-learn.org/stable/datasets/toy_dataset.html). Si presenta desequilibrio entre las clases, mejor. ¿Qué ocurre si es un problema multiclase?